In [ ]:
pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 29.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.34.1
    Uninstalling huggingface-hub-0.34.1:
      Successfully uninstalled huggingface-hub-0.34.1


In [ ]:
pip install peft

In [ ]:
pip install accelerate -U

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "bigscience/bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True
)

In [ ]:
def generate_text(model, tokenizer, prompt_text, max_tokens):
    prompt_text = tokenizer(prompt_text, return_tensors="pt")
    outputs = model.generate(
        input_ids=prompt_text["input_ids"],
        attention_mask=prompt_text["attention_mask"],
        max_length=max_tokens,
        repetition_penalty=1.5,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
initial_output = generate_text(model, tokenizer, "I want you to act as a logistician. ", 100)
print("Initial model output:", initial_output)

In [ ]:
from datasets import load_dataset

dataset_prompt = "fka/awesome-chatgpt-prompts"


data_prompt = load_dataset(dataset_prompt)
data_prompt = data_prompt.map(lambda x: tokenizer(x["prompt"]), batched=True)
train_prompts = data_prompt["train"].select(range(30))

In [ ]:
print(data_prompt['train'][:5])

In [ ]:
import pandas as pd

df = pd.DataFrame(data_prompt['train'][:])

df.head()

In [ ]:
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

tuning_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.RANDOM,
    num_virtual_tokens=4,
    tokenizer_name_or_path=model_name
)

peft_model = get_peft_model(model, tuning_config)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    use_cpu=True,
    output_dir = "./",
    auto_find_batch_size=True,
    learning_rate= 0.005,
    num_train_epochs=5,
    report_to="none",
    )

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_prompts,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


TrainOutput(global_step=20, training_loss=3.6140415191650392, metrics={'train_runtime': 751.5353, 'train_samples_per_second': 0.2, 'train_steps_per_second': 0.027, 'total_flos': 34550802087936.0, 'train_loss': 3.6140415191650392, 'epoch': 5.0})

In [ ]:
tuned_output = generate_text(trainer.model, tokenizer, "I want you to act as a logistician. ", 100)
print("Tuned model output:", tuned_output)

Tuned model output: ['I want you to act as a logistician.  You will be responsible for the logistics of your business.']
